## __Speech → Transcript → Call notes → Send Summary__

This Jupyter notebook aims at transforming my meetings' recordings (audio) into calls notes. 

__→ All you need is: an iPhone (Recording), a Mac (to run this Notebook), working API keys.__

The steps are: 

1. get the transcript from the audio using [Whisper](https://openai.com/index/whisper/) (via [GroqCloud](https://console.groq.com/docs/speech-to-text)) 

2. create a note call from the transcript using [OpenAI](http://openai.com/)

3. send your summary via Gmail

In [ ]:
# SPEECH TO TRANSCRIPT

import os
import io
import time
from pathlib import Path
from dotenv import load_dotenv
from pydub import AudioSegment
from groq import Groq, RateLimitError, InternalServerError

# ── CONFIG ──
load_dotenv()
client = Groq(api_key=os.environ["GROQ_API_KEY"])

DOWNLOADS = Path.home() / "Downloads"
CHUNK_MS   = 60 * 1000   # 60-second chunks
PAUSE_SEC  = 3           # wait 3 seconds between each API call
MAX_RETRIES = 3          # retry up to 3 times on 429 or 503
BACKOFF_BASE = 3         # base backoff seconds for retries (exponential)

for audio_path in DOWNLOADS.glob("*.m4a"):
    print(f"\n→ Loading “{audio_path.name}”…")
    audio = AudioSegment.from_file(audio_path)
    chunks = [audio[i : i + CHUNK_MS] for i in range(0, len(audio), CHUNK_MS)]
    all_text = []

    for idx, chunk in enumerate(chunks, start=1):
        buf = io.BytesIO()
        chunk.export(buf, format="mp3")
        buf.seek(0)

        # Try up to MAX_RETRIES times if we hit 429 or 503
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = client.audio.transcriptions.create(
                    file=(f"{audio_path.stem}_part{idx}.mp3", buf.read()),
                    model="whisper-large-v3",
                    response_format="verbose_json",
                    timeout=120,
                )
                all_text.append(resp.text.strip())
                break  # success → exit retry loop

            except RateLimitError:
                wait = BACKOFF_BASE * attempt
                print(f"429 rate_limit_exceeded (chunk {idx}), retry {attempt}/{MAX_RETRIES} in {wait}s…")
                time.sleep(wait)
                buf.seek(0)
                continue

            except InternalServerError:
                wait = BACKOFF_BASE * attempt
                print(f"503 service_unavailable (chunk {idx}), retry {attempt}/{MAX_RETRIES} in {wait}s…")
                time.sleep(wait)
                buf.seek(0)
                continue

        else:
            # If we exhausted retries, raise an error
            raise RuntimeError(f"Failed to transcribe chunk {idx} after {MAX_RETRIES} attempts.")

        # Regardless of success or after retrying, wait PAUSE_SEC before the next call
        print(f"  → Waiting {PAUSE_SEC}s to respect 20 req/min…")
        time.sleep(PAUSE_SEC)

    # Stitch all chunk transcripts together
    full_transcript = "\n\n".join(all_text)
    out_path = audio_path.with_suffix(".txt")
    out_path.write_text(full_transcript, encoding="utf-8")
    print(f"Saved transcript to {out_path}")


In [ ]:
# TRANSCRIPT to MEETING NOTES
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

for transcript_path in DOWNLOADS.glob("*.txt"):
    print(f"\n→ Processing {transcript_path.name}...")
    
    transcript = transcript_path.read_text(encoding="utf-8")

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.0,
        max_tokens=1500,
        messages=[
        {
            "role": "system",
            "content":
                """
                You are a professional personal assistant that summarizes meeting transcripts into concise, actionable notes.
                
                The transcripts are often long and details, taken during the meetings of Victor Soto (Growth Engineer at UiForm) — those meetings are usually about product, sales, growth, colmpliance, and many other topics encountered in a start-up.
                
                Your task is to provide a detailed and accurate summary of a meeting based on the transcript provided.

                Do not include any personal opinions or interpretations, just the facts as presented in the transcript. 
                Keep all the relevant information, figures, concepts, etc.

                Write your summary in a clear, organized manner that captures the essence of the meeting. Key concepts and figures (sales, pricing, etc.) should be included and easy to find.

                Your tone should be professional and neutral, suitable for business documentation.
                """
        },
        {
            "role": "user",
            "content": f"Please summarize the following meeting transcript:\n\n{transcript}"
        }
        ]
    )
    
    summary = response.choices[0].message.content.strip()
    out_path = transcript_path.with_suffix(".summary.txt")
    out_path.write_text(summary, encoding="utf-8")

    print(f"Summared notes saved to {out_path.name}")

In [ ]:
# SEND NOTES VIA GMAIL

import json
import base64
import markdown2
import re
from email.mime.text import MIMEText

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow


load_dotenv()

BASE_DIR    = Path.cwd()       
CRED_PATH   = BASE_DIR / "credentials.json"
TOKEN_PATH  = BASE_DIR / "token.json"
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def gmail_authenticate():
    creds = None

    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception:
                creds = None

        if not creds:
            if not CRED_PATH.exists():
                raise FileNotFoundError(f"credentials.json not found at {CRED_PATH}")
            flow = InstalledAppFlow.from_client_secrets_file(str(CRED_PATH), SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_PATH, "w") as token_file:
            token_file.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

summary_files = list(DOWNLOADS.glob("*.summary.txt"))
if not summary_files:
    raise FileNotFoundError(f"No files matching “*.summary.txt” found in {DOWNLOADS}")

gmail_service = gmail_authenticate()
to_addr = os.environ.get("TO_EMAIL")
if not to_addr:
    raise RuntimeError("TO_EMAIL environment variable is not set.")

for summary_path in summary_files:
    print(f"\n→ Sending summary for {summary_path.name}…")

    # READ RAW MARKDOWN
    summary_md = summary_path.read_text(encoding="utf-8")

    # CONVERT “**Title**” LINES INTO REAL HEADINGS (## Title)
    summary_md = re.sub(r'(?m)^\*\*(.+)\*\*$', r'## \1', summary_md)

    # MARKDOWN → HTML WITH LINE BREAKS PRESERVED
    summary_html_body = markdown2.markdown(
        summary_md,
        extras=["break-on-newline", "fenced-code-blocks"]
    )

    # WRAP IN YOUR HTML TEMPLATE (BLACK FONT, NO LINES, ETC.)
    html_template = f"""
    <!DOCTYPE html>
    <html>
      <head>
        <meta charset="utf-8">
        <style>
          body {{
            font-family: Arial, sans-serif;
            font-size: 14px;
            line-height: 1.6;
            color: #000;
            margin: 20px;
          }}
          p {{
            margin: 8px 0;
          }}
          ul {{
            margin-top: 4px;
            margin-bottom: 12px;
            padding-left: 20px;
          }}
          li {{
            margin-bottom: 6px;
          }}
          h2 {{
            font-size: 20px;
            margin-top: 24px;
            margin-bottom: 8px;
            color: #000;
            border-bottom: none;
            padding-bottom: 0;
          }}
          h3 {{
            font-size: 16px;
            margin-top: 16px;
            margin-bottom: 6px;
            color: #000;
          }}
          strong {{
            color: #000;
          }}
        </style>
      </head>
      <body>
        {summary_html_body}
      </body>
    </html>
    """

    # BUILD MIME MESSAGE
    message = MIMEText(html_template, "html")
    message["to"]      = to_addr
    message["from"]    = to_addr
    message["subject"] = f"Meeting Summary: {summary_path.stem}"

    # ENCODE & SEND VIA GMAIL
    raw_bytes = base64.urlsafe_b64encode(message.as_bytes())
    raw_str   = raw_bytes.decode()
    gmail_service.users().messages().send(
        userId="me",
        body={"raw": raw_str}
    ).execute()

    print(f"→ Sent: {summary_path.name} as email to {to_addr}.")

print("\n✓ All summary emails have been sent.")

## __WELL DONE!__